In [29]:
%load_ext autoreload
%autoreload 2

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import globals
import data_strings
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
import trajectory_analysis.trajectory_headangle as trajectory_headangle
import data_extraction.get_indices as get_indices
import plotting.plot_probability_chose_wall as plot_probability_chose_wall

### Rewriting of P(choice | visibility) analysis using the logic from opponent-visibility 

In [2]:
data_folder = data_strings.DATA_FOLDER
json_filenames_all_social = data_strings.JSON_FILENAMES_SOCIAL
json_filenames_all_solo = data_strings.JSON_FILENAMES_SOLO

In [3]:
json_filenames = json_filenames_all_social

In [4]:
df, trial_lists = prepare_data.prepare_data(data_folder, json_filenames, combine=False)

filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\first_experiments_2409\240913\2024-09-13_11-31-00_YansuJerrySocial.json
Data is from period before 2024-09-13 00:00:00
Running dataframe through playerinfo_playerposition_conversion.
Loading complete.
Preprocessing complete.
filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\second_experiments_2409\240927\2024-09-27_14-25-20_SaraEmilySocial.json
Loading complete.
Preprocessing complete.
filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\third_experiments_2410\241017\2024-10-17_14-28-40_ShamirAbigailSocial.json
Loading complete.
Preprocessing complete.
filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\fourth_experiments_2410\241017\2024-10-17_16-41-38_ZimoElsaSocial.json
Loading complete.
Preprocessing complete.
filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\fifth_experiments_241112\2024-11-12_13-31-14_KhadijaWendySocial.json
Loading complete.
Preprocessing complete.
filepath: C:\Users\tomha\OneDrive\PhD\SWC\data\sixth_experiments_241112\2024-11-12_15

Goal:
- Parameterise choice wall (if any), first_visible wall (if any), inferred choice

In [5]:
## Create filter funtions


In [6]:
def filter_trials_retrievable_choice(trial_list, player_id, inferred_choice):
    ''' Return the filtered trial list and list of indices from the original trial list that
        conform with player player_id having a recorded choice.
        This is required for accurate probabilities, because we do cannot include trials (as negative)
        where we do not know what the player's choice would have been. '''
    
    # get player choice (wall number) for each trial
    # inferred choice can be used here
    player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                        inferred_choice, debug=False)
    
    print(f"player_choice, inferred status {inferred_choice} is:\n{player_choice}")
    
    # filter trials list to only include trials where this player had a recorded choice
    player_recorded_choice_indices = np.where(~np.isnan(player_choice))

    # index the tuple and allow list-compatible indexing
    player_recorded_choice_indices = player_recorded_choice_indices[0].tolist()

    # Use a list comprehension to index the trial list with the indices list
    trial_list_filtered = [trial_list[i] for i in player_recorded_choice_indices]

    return trial_list_filtered, player_recorded_choice_indices

    

In [7]:
def filter_trials_one_wall_initially_visible(trial_list, player_id, wall_index, current_fov):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to a single trial wall being visible to player player_id at trial start,
        conferred by wall_index (e.g. 0 or 1 for wall1 or wall2) '''
    
    # find wall visibility (at the trial start timepoint) for the full session
    (wall1_visible_session,
    wall2_visible_session) = wall_visibility_and_choice.get_walls_initial_visibility_session(trial_list,
                                                                    player_id, current_fov, debug=False)
    
    # use np bitwise operators to find trials with only the relevant wall visible
    if wall_index == 0:
        single_wall_visible_indices = np.where((wall1_visible_session == True) & (wall2_visible_session == False))
    elif wall_index == 1:
        single_wall_visible_indices = np.where((wall2_visible_session == True) & (wall1_visible_session == False))

    # index the tuple and allow list-compatible indexing
    single_wall_visible_indices = single_wall_visible_indices[0].tolist() 

    trial_list_filtered = [trial_list[i] for i in single_wall_visible_indices]

    return trial_list_filtered, single_wall_visible_indices


In [8]:
def filter_trials_both_walls_initially_visible(trial_list, player_id, current_fov):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to both trial walls being visible to player player_id at trial start. '''
    
    # find wall visibility for the full session
    (wall1_visible_session,
    wall2_visible_session) = wall_visibility_and_choice.get_walls_initial_visibility_session(trial_list,
                                                                    player_id, current_fov,
                                                                    debug=False)
    
    # identify indices where both walls are visible initially
    both_walls_visible_indices = np.where((wall1_visible_session == True) & (wall2_visible_session == True))
    
    # index the tuple and allow list-compatible indexing
    both_walls_visible_indices = both_walls_visible_indices[0].tolist() 

    # Use a list comprehension to index the trial list with the indices list
    trial_list_filtered = [trial_list[i] for i in both_walls_visible_indices]

    return trial_list_filtered, both_walls_visible_indices

In [25]:
def filter_trials_one_wall_becomes_visible_first(trial_list, player_id,
                                                 current_fov, wall_index,
                                                 debug=False):
    ''' Return a filtered trial list and list of indices from the original trial list that
        conform to one of the trial walls becoming visible to player player_id during the trial
        before any other. '''
    
    # get whether given wall is visible first (and initially alone) for this player for a full session
    # TODO This can be checked again after having written
    given_wall_first_visible_session = wall_visibility_and_choice.get_given_wall_first_visible_session(trial_list,
                                                                                                       player_id,
                                                                                                       wall_index,
                                                                                                       current_fov,
                                                                                                       debug)
    
    # identify indices where wall wall_index becomes visible first
    given_wall_first_visible_indices = np.where(given_wall_first_visible_session == True)

    # index the tuple and allow list-compatible indexing
    given_wall_first_visible_indices = given_wall_first_visible_indices[0].tolist() 

    # Use a list comprehension to index the trial list with the indices list
    trial_list_filtered = [trial_list[i] for i in given_wall_first_visible_indices]


    return trial_list_filtered, given_wall_first_visible_indices

In [10]:
def filter_trials_player_chose_given_wall(trial_list, player_id, inferred_choice, given_wall_index):
    ''' Return a filtered trial list and list of indices from the original trial list 
        where player choice (winner + loser, or just winner) aligned with
        the given wall index (e.g., 0 for wall1) '''
    
    # get player choice (wall number) for each trial
    # inferred choice can be used here
    player_choice = wall_visibility_and_choice.get_player_wall_choice(trial_list, player_id,
                                                                        inferred_choice, debug=False)
    
    print(f"player choice array:\n{player_choice}")

    # get the truth array for whether the player choice wall aligns with the given wall parameter
    # to this function (NB. this is NOT the wall that was eventually chosen in the trial)
    given_wall_chosen_session = get_indices.was_given_wall_chosen(trial_list, player_choice,
                                                                  given_wall_index)
    
    print(f"given wall chosen array:\n{given_wall_chosen_session}")
    
    # find the indices of the trials in trial_list where the given wall was chosen by player player_id.
    # this will drop trials where the given wall was not chosen, and trials without retrievable choice information
    given_wall_chosen_indices = np.where(given_wall_chosen_session == True)
    print(f"given wall chosen true indices:\n{given_wall_chosen_indices}")
    
    # index the tuple and allow list-compatible indexing
    given_wall_chosen_indices = given_wall_chosen_indices[0].tolist()

    # use a list comprehension to index the trial list with the indices list
    trial_list_filtered = [trial_list[i] for i in given_wall_chosen_indices]

    # return filtered trial_list, and list of indices with which to index the original list
    return trial_list_filtered, given_wall_chosen_indices
    


In [11]:
def filter_trials_player_won(trial_list, player_id):
    ''' Return a filtered trial list and list of indices from the original trial list 
        where player player_id won. '''
    
    # get trigger activators for this session
    trigger_activators = get_indices.get_trigger_activators(trial_list)
    
    # find the indices of the trials in trial_list where player player_id won
    this_player_wins_indices = np.where(trigger_activators == player_id)
    
    # index the tuple and allow list-compatible indexing
    this_player_wins_indices = this_player_wins_indices[0].tolist()

    # use a list comprehension to index the trial list with the indices list
    trial_list_filtered = [trial_list[i] for i in this_player_wins_indices]

    # return filtered trial_list, and list of indices with which to index the original list
    return trial_list_filtered, this_player_wins_indices
    


In [12]:
## calculate probability

In [13]:
def calculate_probability_choose_wall(trial_list, trial_list_choice_filtered):
    ''' Given a trial list (pre-filtered, but not for choice), calculate the probability that 
        a player will choose a given wall value as the proportion of trials from the trial
        list in which the player chose the wall value.
        More complex use of this function could involve e.g. filtering the trial list for 
        trials where Low was first seen and the Opponent is visible, and then further filtering
        for player choice being 'Low', to find probability of (choose Low | first visible) under the
        condition of Other visibility at trial start. '''
        
    
    # use the length of the trial list pre-choice filtering, and the length of the trial list post-choice
    # filtering (e.g. with filter_trials_player_chose_given_wall) to calculate the proportion of 
    # relevant trials that a player chose a specific wall
    try:
        probability_chose_wall = len(trial_list_choice_filtered)/len(trial_list)
    except ZeroDivisionError:
        probability_chose_wall = np.nan

    return probability_chose_wall
    
    

In [14]:
## Umbrella functions of filters followed by probability calculation

In [15]:
def probability_trial_outcome_given_conditions(trial_list, player_id,
                                               visible_wall_index, chosen_wall_index,
                                               inferred_choice, current_fov,
                                               debug=False):
    ''' Take a trial list and filter based on existence of player_id player choice,
        and whether a specific wall becomes visible alone first (if visible_wall_index is not None). 
        Then, filter this trial list by the player choice being for a specific trial wall 
        (if chosen_wall_index is not None) and take the proportion of trials remaining
        as the probability of making this choice conditioned on our chosen filters.
        Returns probability of choosing a trial wall. 
        inferred_choice decides whether to include inferred choice for identifying retrievable choice and chosen wall.
        visible_wall_index and chosen_wall_index control wall initial visibility and player choice filters respectively. '''
    
    if debug:
      print(f"Next session, player_id {player_id}")

    # filter trials with a retrievable choice for this player (trials only valid for analysis if we 
    # have a recorded choice for the player)
    (trial_list_filtered,
    filtered_indices) = filter_trials_retrievable_choice(trial_list, player_id, inferred_choice)

    if debug:
      print(f"Len 'player choice exists' indices: {len(filtered_indices)}")


    # filter based on single wall visibility if requested
    if visible_wall_index is not None: 
        # filter with visible_wall_index seen first, alone in a trial
        (trial_list_filtered,
        filtered_indices) = filter_trials_one_wall_becomes_visible_first(trial_list_filtered, player_id,
                                                                     current_fov, wall_index=visible_wall_index)
    if debug:
      print(f"Len '{visible_wall_index} seen first, alone': {len(filtered_indices)}")


    # filter based on chosen wall if requested
    if chosen_wall_index is not None:
        # filter with chosen_wall_index chosen
        (trial_list_filtered_choice,
        filtered_indices_choice) = filter_trials_player_chose_given_wall(trial_list_filtered, player_id,
                                                                  inferred_choice,
                                                                  given_wall_index=chosen_wall_index)
    else: # otherwise, default to all wins for this player
        (trial_list_filtered_choice,
        filtered_indices_choice) = filter_trials_player_won(trial_list_filtered, player_id)
       
    if debug:
      print(f"Len '{chosen_wall_index} index wall chosen by player {player_id}': {len(filtered_indices_choice)}")

    
    # find the probability of the player choosing the given wall index, considering only trials that are filtered
    # prior to choice filtering
    probability_chose_wall = calculate_probability_choose_wall(trial_list_filtered, trial_list_filtered_choice)

    if debug:
      print(f"Probability player chose wall given these conditions: {probability_chose_wall}")

    
    return probability_chose_wall

In [16]:
## Session umbrella functions for collecting probabilities data for all players in all sessions

In [23]:
def probability_trial_outcome_given_conditions_all_sessions(session_trial_lists, inferred_choice,
                                                            current_fov,
                                                            chosen_wall_index, visible_wall_index,
                                                            trial_type=globals.HIGH_LOW, debug=False):
    ''' Return the probability array across all players and sessions 
        that a trial has a particular outcome given conditions.
        Conditions here are only wall visible_wall_index visible alone, first.
        Outcome is that wall chosen_wall_index is chosen, or just that player player_id
        won if this parameter is None'''

    probabilities = np.full(len(session_trial_lists)*2, np.nan)
    trial_list_index = 0

    # loop through the number of players, filling 2 slots per session analysed
    for i in range(0,len(session_trial_lists)*2, 2):

        trial_list = session_trial_lists[trial_list_index]

        # filter trial list for HighLow trialtype
        trial_indices = get_indices.get_trials_trialtype(trial_list, trial_type=trial_type)
        trial_list_filtered = [trial_list[i] for i in trial_indices]

        for player_id in [0,1]:
            probabilities[i + player_id] = probability_trial_outcome_given_conditions(trial_list_filtered, player_id,
                                                                                      visible_wall_index, chosen_wall_index,
                                                                                      inferred_choice, current_fov,
                                                                                      debug=debug)

        trial_list_index += 1
        
        

    return probabilities


In [18]:
## Plotting functions for boxplot line graphs

In [19]:
# plot_probability_chose_wall.boxplot_probability_choose_wall(wall_choice_probabilities, wall_choice_labels, ylabel="P(Choose first wall seen)")

### Sandbox

In [20]:
# variables
inferred_choice = False
current_fov = 110


In [28]:
# P(H | H first vis), for all players
probabilities1 = probability_trial_outcome_given_conditions_all_sessions(trial_lists, inferred_choice,
                                                                        current_fov, trial_type=globals.HIGH_LOW,
                                                                        chosen_wall_index=0, visible_wall_index=0,
                                                                        debug=False)

player_choice, inferred status False is:
[nan  2.  5.  6. nan nan  5. nan  8. nan nan  6. nan nan nan nan  1. nan
  1. nan nan  2. nan  8. nan  5.  2.  5. nan  3. nan nan nan nan nan  2.
  7. nan nan nan  6.  4.  2.  6. nan  6.  1.  7. nan nan  6. nan  3. nan
 nan nan nan nan  3. nan  1.  4.  7. nan  8.  2.  1.  1. nan nan nan nan
 nan  7. nan nan  1. nan  1.  8. nan nan  5. nan nan  2. nan nan nan nan
 nan nan nan  1.  8.  4. nan nan nan nan nan  7.]


TypeError: vstack() takes 1 positional argument but 2 were given

In [ ]:
# P(L | L first vis), for all players
probabilities2 = probability_trial_outcome_given_conditions_all_sessions(trial_lists, inferred_choice,
                                                                        current_fov, trial_type=globals.HIGH_LOW,
                                                                        chosen_wall_index=1, visible_wall_index=1,
                                                                        debug=False)

In [ ]:
wall_choice_probabilities = [probabilities1, probabilities2]
wall_choice_labels = ["High first vis", "Low first vis"]

In [ ]:
plot_probability_chose_wall.boxplot_probability_choose_wall(wall_choice_probabilities, wall_choice_labels, ylabel="P(Choose first wall seen)")